<style>
.jp-RenderedHTMLCommon pre {
        background-color: #FFFFF0FF;
    }
</style>

# Distributed Systems
## 2021/22

Lab 3

Nuno Preguiça, Sérgio Duarte, Dina Borrego, João Vilalonga

# Goals

In the end of this lab you should be able to:

+ Know how to deal with errors on REST clients;
+ Know how to track errors on REST servers;
+ Know how to test your 


## REST - Client Errors

## Causes

A REST request might fail for several reasons:

+ The server is not running;
+ The server is slow;
+ A TCP connection was dropped;
+ The network failed;
+ There was a network anomaly (e.g., routing)


## Transient failures

Temporary failures can be ***masked*** by issuing
the request multiple times.

Usually, the client quits after a few retries to
avoid blocking the application forever,<br>for example, in
case the server has crashed.

Note:

Transient failures are temporary issues that resolve themselves shortly.

## ProcessingException

Jersey (JAX-RS) exposes request failures to clients in the form of a Java exception: [javax.ws.rs.ProcessingException](https://docs.oracle.com/javaee/7/api/javax/ws/rs/ProcessingException.html)

A **try{ } catch{}** block can be used to retry the request automatically after a small amount of time.


Note: Waiting a bit before retrying the request prevents a too agressive client behavior and allows some time for the transient error condition disappear.

### Example - CreateUser (1)

Setting the client timeout values...

```java
protected static final int READ_TIMEOUT = 5000;
protected static final int CONNECT_TIMEOUT = 5000;

ClientConfig config = new ClientConfig();

config.property(ClientProperties.READ_TIMEOUT, READ_TIMEOUT);
config.property( ClientProperties.CONNECT_TIMEOUT, CONNECT_TIMEOUT);
		
Client client = ClientBuilder.newClient(config);

```

In [ ]:
In the future, other changes to client behavior will be done the someway.

### Example - CreateUser (2)
```java
protected static final int MAX_RETRIES = 10;
protected static final int RETRY_SLEEP = 1000;

@Override
public String createUser(User user) {

	WebTarget target = client.target( serverURI ).path( RestUsers.PATH );
	for (int i = 0; i < MAX_RETRIES; i++)
		try {
			Response r = target.request()
				.accept(MediaType.APPLICATION_JSON)
				.post(Entity.entity(user, MediaType.APPLICATION_JSON));

            if( r.getStatus() == Status.OK.getStatusCode() && r.hasEntity() )
                // SUCCESS
                return r.readEntity(String.class);
            else {
                System.out.println("Error, HTTP error status: " + r.getStatus() );
                break;
            }
		} catch (ProcessingException x) {
            sleep( RETRY_SLEEP );
        }
	return null; // Report failure
}
```

### Can we do better?

The sample code above needs to be **repeated for all operations** of all services!

 + Doable but **error prone** because it invites a lot of ***cut & paste***...

Can we make it more general?

Of course!!!

### Step 1 - Implement the request as a private method

```java 

private String clt_createUser(User user) {
    Response r = target.request()
                    .accept(MediaType.APPLICATION_JSON)
                    .post(Entity.entity(user, MediaType.APPLICATION_JSON));

    if( r.getStatus() == Status.OK.getStatusCode() && r.hasEntity() )
        return r.readEntity(String.class);
    else
        return null;
}
```

### Step 2 - Implement the retry behavior as a generic operation

```java
protected <T> T reTry(java.util.function.Supplier<T> func) {
    for (int i = 0; i < MAX_RETRIES; i++)
        try {
            return func.get(); // Success
        } catch (ProcessingException x) {
			sleep(RETRY_SLEEP);
		} catch (Exception x) {
            // Handle other errors
            break;
        }   
    return null; // Failure
}
```

Note:
    
This method can be part of super class inherited by all service clients.

### Step 3 - Implement the requests with retry behavior

```java
public String createUser(User user) {
    return reTry( () -> clt_createUser( user ));
}

public User getUser(User user, String password) {
    return reTry( () -> clt_getUser( user, password ));
}

```

We are making use of [Java Lambda Expressions](https://docs.oracle.com/javase/tutorial/java/javaOO/lambdaexpressions.html)...

`() -> clt_createUser( user )` is a function that returns a result, making it compatible with the functional interface [Supplier&lt;T&gt;](https://docs.oracle.com/javase/8/docs/api/java/util/function/Supplier.html) used as the parameter of the `reTry` generic method.



# Tracking REST server-side errors


### Unhandled server-side exceptions

Jersey reports to clients any unhandled exception thrown in the scope of 
an implementation method of a service as `500 Internal Error`.

The default Jersey runtime behavior is to suppress the actual exception details, including type or stack trace,
making it very difficult to diagnose the source of the problem.

### CustomExceptionMappers

Jersey provides a way to customize how server-side exceptions are reported before a response is sent to the client.

For instance, the class below is a custom exception mapper that will allow the stack trace
of an exception to show up on server logs.

```java
public class GenericExceptionMapper implements ExceptionMapper<Throwable> {
	@Override
	public Response toResponse(Throwable ex) {

		if (ex instanceof WebApplicationException wex) {
			Response r = wex.getResponse();			
			if( r.getStatus() == Status.INTERNAL_SERVER_ERROR.getStatusCode())
				ex.printStackTrace();
			return r;
		}
		ex.printStackTrace();
		return Response.status(Status.INTERNAL_SERVER_ERROR)
            .entity(ex.getMessage()).type(MediaType.APPLICATION_JSON).build();
	}
}
```

The code will still report the error to the client as `500 Internal Error`.

However, it also prints the exception stack trace to the server output, making it easier to find 
which class, method and line of code caused the problem.

### Registering CustomExceptionMappers

To use a custom exception mapper, we have register it in addition to the services exposed
by the server, like so:

```java
        ResourceConfig config = new ResourceConfig();
		config.register(UsersResource.class);
		config.register(GenericExceptionMapper.class);

        String ip = InetAddress.getLocalHost().getHostAddress();
		String serverURI = String.format(SERVER_URI_FMT, ip, PORT);
		JdkHttpServerFactory.createHttpServer( URI.create(serverURI), config);
	
```

# Concurrency Issues in the Server

### Multi-threaded services

For performance reasons, servers are rarely single-threaded.

Concurrent client requests are usually handled in separate threads.

If two or more threads access **shared state** without any form of 
concurrency control,<br>the behavior of the application can be anomalous, unpredicatable or faulty.

#### Example anomaly

This code can lead to missed updates...

```java
private final Map<String, Integer> counters = new HashMap<>();

void updateCounter(String counterId, int increment) {
    Integer currentValue = counters.get( counterId );
    if( currentValue == null )
        counters.put( couterId, increment ) ;
    else
        counters.put( counterId, currentValue + increment);
}
```



For instance, `currentValue` can be evaluated to `null` by multiple threads, or 
they can retrieve the same previous value and apply their increment to that value.
On both occurrences, some increments will be lost.

Moreover, the [HashMap](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/HashMap.html) is documented as not being thread-safe. This is common for most of the data structures in the [java.util]([HashMap](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/package-summary.html).

### Java Concurrency Control

Java provides some mechanisms for concurrency control...

+ Thread-safe data-strutures
+ Synchronization

### Single-object concurrency control

Concurrent requests involving single objects can sometimes be handled
by leveraging thread-safe data-structures, such as:

[ConcurrentHashMap](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/concurrent/ConcurrentHashMap.html)
[CopyOnWriteArrayList](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/concurrent/CopyOnWriteArrayList.html)

For single variables the need atomic updates, such as increments:

[AtomicInteger](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/concurrent/atomic/AtomicInteger.html)
[AtomicLong](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/concurrent/atomic/AtomicLong.html)
[AtomicReference](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/concurrent/atomic/AtomicReference.html)



Check package [java.util.concurrent]((https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/concurrent/package-summary.html) for a variety of classes that replace common data-structures found in [java.util]([HashMap](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/package-summary.html) with thread-safe counterparts.

Performance of thread-safe data-structures is often fine-tuned for specific scenarios. Read the documentation. 

### Multi-object concurrency control

In Java, synchronization is the simplest way to handle concurrency issues
that involve inter-related objects, <br> as well as single-objects.

The `synchronized` keyword allows a ***block of code*** or a ***method*** to execute
sequentially - a thread at a time.

```java
synchronized(obj) {
     // sequential code
}
```
```java
synchronized public void createUser( User user) {
    // sequential code
}
```
```java
public void createUser( User user) {
    synchronized(this){
    // sequential code
    }
}
```

Java native [synchronization](https://docs.oracle.com/javase/tutorial/essential/concurrency/sync.html) requires an object to act as a *monitor*. 

For synchronized instance (object) methods, ***this*** is the implicit object used as the monitor. For synchronized class methods (static), it is the ***class***. This means that synchronized methods of the same object (or class) are all mutually exclusive and run sequentially.

### Impact on performance

Concurrency control has a negative impact on performance.

For instance, using synchronization to prevent *write-write* and *read-write* conflicts,<br>
will also lead to read-only operations to run sequentially, when they usually do not interfere.

For finer concurrency control, Java also provides higher-level constructs, such as [Locks](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/util/concurrent/locks/package-summary.html).

# Exercises

1. Test the retry mechanism to mask transient failures
2. Test the mechanism for exposing server-side errors

## Test masking transient failures


1. Download this [lab's project](http://preguica.github.io/sd2122/code/sd2122-lab3.zip);
2. Build the Docker image, using the usual maven command;

  `mvn clean compile assemby:single docker:build`
3. Launch the server;

 `docker network create -d bridge sdnet` (if necessary)
 
 `docker run --rm -h users-1 --name users-1 --network sdnet -p 8080:8080 sd2122-aula3-xxxxx-yyyyy`
 
4. Create the client container:

    `docker run -it --network sdnet sd2122-aula3-xxxxx-yyyyy /bin/bash`

5. Create a new user.

    In the client container shell, type:

    `java -cp /home/sd/sd2122.jar sd2122.aula3.clients.CreateUserClient http://users-1:8080/rest nmp "Nuno Preguica" nmp@nova.unl.pt 12345`
    
    Confirm the request succeeded and returned: `nmp`
    
6. Stop the server.

    CTRL-C or execute in another terminal:
    
    `docker rm -f users-1`
    
7. Execute step 5 again.

   This time, since the server is not running, the client should output:
   
   `FINE: ProcessingException: java.net.UnknownHostException: users-1`
   
8. Launch the server again (step 3)

   The client should finish and return: `nmp`
   

## Test exposing server-side errors


1. Download this [lab's project](http://preguica.github.io/sd2122/code/sd2122-lab3.zip) (if necessary);

2. In the main method of the `UsersServer.java` file, uncomment the following line:

   `config.register(GenericExceptionMapper.class);`

3. Build the Docker image, using the usual maven command;

  `mvn clean compile assemby:single docker:build`
  
4. Launch the server;
 
 `docker network create -d bridge sdnet` (if necessary)
 
 `docker run --rm -h users-1 --name users-1 --network sdnet -p 8080:8080 sd2122-aula3-xxxxx-yyyyy`
 
5. Create the client container:

    `docker run -it --network sdnet sd2122-aula3-xxxxx-yyyyy /bin/bash`
       
6. Try the SearchUsersClient:
    
    In the client container shell, type:

   `java -cp /home/sd/sd2122.jar sd2122.aula3.clients.SearchUsersClient http://users-1:8080/rest nmp`
   
   The client should report:
   
   `Error, HTTP error status: 500`
   
   In the server, the top line of stack trace shown should be:
   
   `at sd2122.aula3.server.resources.UsersResource.searchUsers(UsersResource.java:96)`

   This is the *method*, *class file* and *line number* where the uncaught exception was thrown.

7. Undo Step 2 (by commenting the line); 

8. Repeat Steps 3, 4, 5 and 6.

   This time the output of the server will only show that the request reached the server. It will not
   show that an exception was thrown while processing the request. 
   